STA 663 Final Project

Xinghong Tang, Jingyi Zhang

----------------------------------------
# Latent Dirichlet Allocation
## 1 Abstract
Latent Dirichlet Allocation (LDA) is a generative statistical model for discrete collections of data. It is a multi-level hierarchical Bayesian model, in which each item of a collection is modeled as a finite mixture over an underlying set of topics, thus is also called a topic model.

In this project, we will first discuss the mathematical and statistical background of LDA, then implement the algorithm and functionalities of LDA with Python, illustrate the package with some text examples and finally improve the performance of the package through optimization methods.

## 2 Background
### 1 Problem Addressed by LDA
When we read a news magazine, say about fashion, there are certain topics that we expect to see from articles in this magazine. Within each article, depending on the theme or topic, some words are moree likely to appear than others. From a different perspective, if we were to write a paper on an issue, there are certain related topics to cover, and for these topics we would much more strongly prefer vocabulary that is more related to the topic chosen than some other irrelevant ones.

LDA model here follows the same logic: for a certain document In the context of machine learning and semantic analysis, LDA is a model that processes text documents as a discrete collections of data. From top to bottom levels, we define the text collections to be "corpus," a collection of documents,"document," a collection of words, and lastly the lowest basic unit, "words."

Here we focus on the functionalities of LDA in text mining. Under the assumption of exchangability (orders of documents in a corpus and orders of words in a document can be neglected), the main purposes of LDA are to find, under a corpus, 1. the distribution of topics within each document; 2. the distribution of words within each topic.


### 2 Application
LDA is now widely used in not only linguistics and semantic analysis fields but also other sciences. Its original and most natural usage is, of course, processing and analyzing collections of text documents. Other than that, LDA is also used in genetic and ancestral tracking. 

One example of these extended applications is that, some popular services that offer to track the ancestry of a person with his or her DNA is actually an application of LDA. Instead of analyzing texts with natural language, LDA considers, in the context of genetic analysis, scientists look at the distribution of all human ancestries (the entire human ancestry collection is analogous to a "document," thus each one corresponds to a "topic"). Possible genetic segments under each ancestry can be considered as the "words." Thus, from a person's DNA, the "words" could be traced back and then could find out the most likely "topics," in this context ancestries, that it comes from.

### 3 Other Similar Algorithms
Remain in the context and applications of natural language processing, algorithms similar to the functionalities and purposes of LDA include: unigram model, miture of unigram models, probablistic Latent Semantic Indexing (pLSI), and probablistic Latent Semantic Analysis (pLSA).

Comparing to these models, LDA has the following advancement and advantages:

- LDA allows documents, instead of assuming one topic per document, to present multiple topics within the same document with minor cost (comparing to unigram and mixture of unigram models);

- LDA is a well-defined generative model and can assign probability to both seen and unseen data. Since no indexing is involved, LDA doesn't have the issue of linearly growing number of parameters as number of topic increases, thus sparing the concern of overfitting (comparing to pLSI).

- LDA has better generalization performance than the others. As shown in section 7 of Blei et al. (2003), the perplexity score, predictive perplexity and accuracy of LDA are generally better than unigram, mixture of unigram and fold in pLSI models.

- LDA is more of a Bayesian approach while pLSA is more on the frequientist side. Because of this fundamental difference in the initial approach to the problem of interest, the output of the methods may be different, even though both are relatively advanced models.

## 3 Algorithm & Mathematics
### 1 Bayesian Statistics
To help understand the following hierarchical structure and the later sampling/variational inferences, we briefly demonstrate the Baye's theorem and its interpretation in terms of distributions.

$\textbf{Bayes's theorem}:\\
P(A|B) = \frac{P(B|A)P(A)}{P(B)} \to P(A|B) \propto P(B|A)P(A)$

Here, A is the parameter of interest and B is the data. Thus, P(A) is called the prior distribution, which is our prior knowledge or belief of the parameter before seeing the data; P(B|A) is the likelihood distribution, which is the likelihood of seeing the data; and lastly, the product of the prior and the likelihood is proportional to, thus yields, the possterior distribution of the parameter P(A|B), which is an updated distribution of the parameter after seeing and weighing in the data.

$\to \textbf{Posterior} \propto \textbf{Prior} \times \textbf{Likelihood}$

### 2 Hierarchical Modeling: Dirichlet & Multinomial
#### Algorithm in Plain Language
The The essense of LDA is a hierarchical model. To accomplish the final goal of assessing both the distribution of topics within each document and the distribution of words within each topic.

Same as the logic of human writing a collection of articles of the same theme, we would have some fixed expected topics under the theme, and for each topics, there are certain words to be expected to appear. With the same logic, we develop the following hierarchical model system:

- First we assume the topics (T's) in a document come from a distribution A;
- We also assume the words in some topic T follows a distribution B;
- We repeat the following steps to obtain words for the article:
    - Get a topic T from A;
    - From T, we get words from the corresponding distribution B.
    
The mathematical explanation with clearer notations are shown below.

#### Mathematical Explanation
Before introducing the modeling algorithm, the following are the probability density functions of the distributions involved, Dirichlet and Multinomial distributions:
- Dirichlet Distribution:

$Dirichlet(p|\alpha) = \frac{\Gamma (\sum_{k=1}^K \alpha_k)}{\prod_{k=1}^K \Gamma(\alpha_k)}\prod_{k=1}^K p_k^{\alpha_k-1}$

- Multinomial Distribution:

$Multinomial(m_1, m_2, m_3|n,p_1,p_2,p_3) = \frac{n!}{m_1!m_2!m_3!}p_1^{m_1}p_2^{m_2}p_3^{m_3}$

- Conjugate Posterior:
$Dirichlet(p|\alpha) \times Multi(m) \propto Dirichlet(p|\alpha+m)$

The following shows statistical modeling algorithm behind LDA:

$\textbf{Notations:}$

K: number of topics;

V: number of words in the entire vocabulary;

M: number of documents;

$N_d$: number of words in document $\textit{d}$;

$\alpha_k$: prior weight of topic k in a document;

$\beta_v$: prior weight of word $\textit{w}$ in a topic;

$\varphi_k$: probability distribution of words in topic $\textit{k}$;

$\theta_d$: probability distribution of topics in document $\textit{d}$;

$z_{d=1,...,M,w=1,...,N_d}$: identity of topic of word $\textit{w}$ in document $\textit{d}$;

$w_{d=1,...,M,w=1,...,N_d}$: identity of word $\textit{w}$ in document $\textit{d}$.

$\textbf{Generative Process:}$

- Choose $\theta_i \sim Dir(\alpha)$, which is the prior belief of the topic distributions under documents;
- Choosee $\varphi_k \sim Dir(\beta)$, which is the distribution of words under topic k;
- For each word i,j (jth word in the ith document):
    - Choose a topic $z_{i,j} \sim Multinomial(\theta_i)$;
    - Choose a word $w_{i,j} \sim Multinomial(\varphi_k)$.
With the actual training data, we will be able to update the values of $\alpha$ and $\beta$ to find the desired distributions. However, the posterior distributions are not closed form with simple parameter updating formula, thus we need to use other methods (sampling or variational inference) to approximate the parameters.

##### Here, we choose variational inferencing as the initial approach for the approximation, and then use Gibbs Sampling as a better algorithm to optimize the model.

In the following section, the approximation calculations/steps are introduced.

### 3 Gibbs Sampling & Variational Inference
#### (a) Variational Inference
Expectation-maximization (EM) is mainly used to provide a parameter optimization framework for maximum likelihood estimation (MLE), i.e. maximizing $L(\theta) = p(X;\theta)$. 

For simple problems such as estimating the $\mu$ and $\sigma$ for multivariate Gaussian distribution, it is easy to get the estimates by taking partial derivatives of parameters. However, when the distribution of X does not depend on $\theta$, but on some other random hidden variables Z, the estimtation gets complicated. Because $log(x)$ is a function that is monotonically increasing, we could equivalently optimize log likelihood $l(\theta) = logp(X;\theta)=log\int p(X,Z = z;\theta)dz$. EM algorithm is aimed at this scenario with the theoretical basis of Jensen's Inequality. 

Based on Jensen's Inequality, starting from some initial point, EM uses a method similar to coordinate ascent method to optimize log likelihood until it reaches a local extremum. Each iteration in this algorithm can be decomposed into two steps: E-step and M-step. Find the expectation value in E-step and Optimize the expectation value in M-step. 

Variational Bayes methods is a common approximate inference method for posterior inference. It aims to solve the problem that a posterior distribution $p(Z=z|X)$ of a hidden variable Z does not have a simpflied form. A simple way to approximate the posterior distribtuion is to use a parameterized distribution $q(z;\theta)$ to fit $p(Z=z|X)$. As a measurement of effect, we expect to find $\theta^{*}$ to minimize KL divergence between p and q. We know that $\int q(z)log\frac{p(X,Z=z)}{q(z)}dz = -KL(Q|P(\cdot |X))+logp(X)$. If we assume the form of $q(Z;\theta)$, then minimizing $KL(Q|P(\cdot |X))$ is equivalent to maximizing the lower bound $E_{Z\sim Q}[log\frac{p(X,Z)}{q(Z;\theta))}]$. Only here we fix the form of q. 

For LDA, we use mean field assumption for variazation inference that all latent variables are independent of each other so that $q(Z;\theta)$ is the product of probability density functions of latent variables. After we get the Lower Bound $L(\gamma,\phi;\alpha,\beta)$,we maximize it with respect to variational parameters($\gamma,\phi$) to get q in E-step and fix the variational parameters and maximize the Lower Bound with respect to model parameter ($\alpha, \beta$).
#### (a1) EM Algorithm: E-Step
$\phi_{nk} \propto (\sum_{i=1}^{V} w_n^i(\Psi(\lambda_{ki})-\Psi(\sum_{i=1}^{V}\lambda_{ki}))+\Psi(\gamma_k)-\Psi(\sum_{k=1}^{K}\gamma_{k}))$<br>
$\gamma_k = \alpha_k + \sum_{n=1}^{N_d}\phi_{nk}$ <br>
$\lambda_{ki} = \beta_i+\sum_{d=1}^M\sum_{n=1}^{N_d}\phi_{dnk}w_{dn}^i$
#### (a2) EM Algorithm: M-Step
$L_{[\alpha]} = \sum_{d=1}^M(log\Gamma(\sum_{k^{'}=1}^K\alpha_{k^{'}})-\sum_{k=1}^{K}log\Gamma(\alpha_k)+\sum_{k=1}^{K}(\alpha_k-1)(\Psi(\gamma_{dk})-\Psi(\sum_{k^{'}=1}^K\gamma_{dk^{'}}))$<br>
$\frac{\partial L}{\partial \alpha_k}=M(\Psi(\sum_{k^{'}=1}^K\alpha_{k^{'}})-\Psi(\alpha_k))+\sum_{d=1}^{M}(\Psi(\gamma_{dk})-\Psi(\sum_{k^{'}}^K\gamma_{dk^{'}}))$
#### (b) Gibbs Sampling
With the conjugacy nature stated above, it is not hard to derive a Gibbs sampling system to update and thus estimate the parameters from the data that we work with. The sampling steps for each cycle of updating are as following, with the full conditionals:

- $p(z_{ij} = k|\theta_i,\varphi_k) \propto exp(log \theta_{ik} + log \varphi_{k,w_{ij}})$

- $p(\theta_i|z_{ij} = k, \varphi_k) = Dir(\alpha+\sum_l I(z_{il} = k))$

- $p(\varphi_k|z_{ij} = k, \theta_i) = Dir(\beta+\sum_i \sum_l I(z_{il} = k,w_{il} = j))$

## 4 Usage in Researches
The LDA model is a very powerful tool for processing natural language and other similar types of documents. It can be a great aid for linguistic, semantic, genetic and other research fields.
LDA model can be used to make inferences on the distributions of topics and words in documents and corpra. Because of the complexity of natural language, the existense of this model can greatly increase the accuracy and efficiency in summarizing text files. Moreover, it is also able to make predictions for documents with missing segments, which can be extremely helpful for reasearches on ancient languages, damaged text documents and etc,.

## 5 References